In [13]:
import pandas as pd

In [14]:
mc = pd.read_csv('./out_member/member_churn_feb_test3.csv', dtype='object')
mc.columns = ['no_member','pred']
mc = mc[['no_member']]

In [15]:
spath = '/home/server/gli-data-science/akhiyar/churn/out_member/target_churn_edit.xlsx'
adder = 'Sheet1'
writer = pd.ExcelWriter(spath, engine='xlsxwriter') 
mc.to_excel(writer, sheet_name=adder, index=False, na_rep='NaN')

# Auto-adjust columns' width
for column in mc:
    column_width = max(mc[column].astype(str).map(len).max(), len(column)) + 5
    col_idx = mc.columns.get_loc(column)
    writer.sheets[adder].set_column(col_idx, col_idx, column_width)

writer.save()

In [ ]:
df_tto_12 = df_tto[df_tto['TRO_DATE_MONTH'] == '2021-12']
df_tto_12_g = df_tto_12.groupby(['TRO_MEMBERS']).agg(
                    SALES=('TRO_NET','sum'),
                    STRUK=('TRO_NO_ORDER', 'nunique'),
                    PLU=('TRO_PLU', 'nunique')).reset_index()

df_tto_12_g['STRUK/PLU'] = df_tto_12_g['STRUK'].astype(float) / df_tto_12_g['PLU'].astype(float)


q3 = np.quantile(df_tto_12_g['SALES'], 0.75)
q1= np.quantile(df_tto_12_g['SALES'], 0.25)

avgq_sales = q3 + (1.5 * (q3-q1))
print(avgq_sales)


q3 = np.quantile(df_tto_12_g['STRUK'],0.75)
q1= np.quantile(df_tto_12_g['STRUK'],0.25)

avgq_struk = q3 + (1.5 * (q3-q1))
print(avgq_struk)

q3 = np.quantile(df_tto_12_g['STRUK/PLU'], 0.75)
q1= np.quantile(df_tto_12_g['STRUK/PLU'], 0.25)

avgq_struk_plu = q3 + (1.5 * (q3-q1))
print(avgq_struk_plu)


df_tto_12_g['AB_SALES'] = np.where(df_tto_12_g['SALES'] > avgq_sales, 1, 0)
df_tto_12_g['AB_STRUK'] = np.where(df_tto_12_g['STRUK'] > avgq_struk, 1, 0)
df_tto_12_g['AB_STRUK/PLU'] = np.where(df_tto_12_g['STRUK/PLU'] > avgq_struk_plu, 1, 0)

df_tto_12_g['WEIGHT'] = df_tto_12_g['AB_SALES'] + df_tto_12_g['AB_STRUK'] + df_tto_12_g['AB_STRUK/PLU']

writer = pd.ExcelWriter('./member_weight.xlsx', engine='xlsxwriter')

##################
count_avg = df_tto_12_g[df_tto_12_g['WEIGHT'] >= 2].copy()
count_avg.to_excel(writer, sheet_name='sheet1', index=False)

# Auto-adjust columns' width
for column in count_avg:
    column_width = max(count_avg[column].astype(str).map(len).max(), len(column)) + 4
    col_idx = count_avg.columns.get_loc(column)
    writer.sheets['sheet1'].set_column(col_idx, col_idx, column_width)
    
writer.save()